Joins
* Inner Joins

          Equi Joins
          Non-Equi Joins
          Self Joins
* Outer Joins
  
          Left Outer Joins
          Right Outer Joins
          Full Outer Joins
* Cross Joins

#### Inner Joins
Combines records from 2 tables whenever there are matching values in a field common to both tables (returns only those rows that satisfy the joining condition)

1. Equi Join: An Equi Join is a joining condition with equality operator 

2. Non-Equi Join: An Non-Equi Join is a joining condition with anything other than equality operator (say between and operator)

3. Slef Join: This is an Equi Join of a table with itself 

After joining the data in ORM, the resulting table structure look like shown below:

        EmpNo   EName   Job   Sal   Mgr   Comm   HireDate    DeptNo           DeptNo     
                                                                      DName   Loc   DeptNo
        123      ABC    X     345   21     22    2000-09-09   15      JKL      XY    56
        124      BCA    X     344   21     18    2000-05-10   15      JKL      XY    56
        125      CAB    X     343   21     20    2000-07-05   15      JKL      XY    56

Syntax for accessing child table in Front End:

                    object_name.column_name
Syntax for accessing parent table data in Front End:

                    object_name.common_column.column_name

#### Note
* While writing the condition for Child table column, we can directly provide

        column_name = 'value'
* While writing condition based on Parent table column, we should use below mentioned syntax

        commonclm_name__Parent_colName = 'value'
  

#### prefetch_related

                  select_related                                                        prefetch_related
    1. One-to-One & One-to-Many (FK)                                           1. Many-to-one & Many-to-Many 
    2. We are joining child table to parent table                              2. We are joining parent table to child table
    3. We use common column for writing joining condition                      3. We use related_name for writing joining condition
    4. Structure of table:                                                     4. Strucutre of table: 
    Child_CN1 Child_CN2.....Child_CNn Parent_CN1 Parent_CN2....Parent_CNn      Parent table object_1
        --       --    .....        --       --          --     ....   --              all child table data
                                                                               Parent table object_2
                                                                                       all child table data

* views.py

      def DeptEmp_prefetchRelated(request):
        LEDO = Dept.objects.prefetch_related('emp_set').all()
        d = {'LDEO':LEDO}
        return render(request, 'DeptEmp_prefetchRelated.html', d)

* related_name has 2 functions:

      1) as argument for prefetch_related function for joining Parent table to Child table

                  default value is "childTableName_set"
         Note that childTableName in related_name is always in the lower case irrespective of its original case
                  
      2) fetching the child table data without using child table name and only using parent table name
         Syntax is

                  ParentTableObject.related_name.all
      

#### In prefetch_related --> data is fetched through the parent table to child table
* syntax for filtering based on Parent Table Column
  
        PTN.objects.prefetch_related('related_name').filter(PColName = value )

        Here the specified PTO (Parent Table Objects) and all the related CTO (Child Table Objects) are fetched         

* syntax for filtering based on Child Table Column to get all PTO but only specified CTO

        PTN.objects.prefetch_related(Prefetch('related_name', queryset = CTN.objects.filter(CColName = value )))

* syntax for filtering based on Child Table Column to get specified PTO and only related specified CTO

        PTN.objects.filter(PCN = value).prefetch_related(Prefetch('related_name', queryset = CTN.objects.filter(CColName = value )))


#### Examples
* Query to fetch all the Department objects and all of its related Child Table Objects
  
        LEDO = Dept.objects.prefetch_related('emp_set').all()
* Query to fetch only Dept 10 object and only all of its related Emp objects 

        LEDO = Dept.objects.prefetch_related('emp_set').filter(Dept_No = 10)

* Query to fetch all the Department objects and only Raj Emp object

        LEDO = Dept.objects.prefetch_related(Prefetch('emp_set', queryset=Emp.objects.filter(EName = 'Raj')))

* Query to fetch only the Department 10 object and only Raj Emp object

      DN = Dept.objects.filter(EName = 'Raj').values_list('Dept_No)  
      LEDO = Dept.objects.filter(Dept_No__in = 10).prefetch_related(Prefetch('emp_set', queryset=Emp.objects.filter(EName = 'Raj')))
        


values -----> returns a queryset of list of dictionaries with key as column names and values as their values

values_list  ----> returns a queryset of list of tuples with values of  the specified column names in the condition

#### aggregate 
* aggregate function can be used for performing all the aggregation procedures on the query set 
* Some of the aggregate methods are Max, Min, Sum, Avg, Count etc.
* Syntax:

        from django.db.models import Max, Min, Sum, Avg, Count
        Max_sal_dict = Emp.objects.aggregate(Max('Sal'))
* By default, the aggregate method return a dictionary with key as the column name and lowercase of specified aggregate function name combined using double underscore.

        Eg:'Sal__max' in the given case 
* But we can give custom key names by specifying the desired keyname within the aggregate function and value as the specified aggregate function

        Eg: aggregate(emp_max_sal = Max('Sal'))
* We can fetch the resulting aggregate function value by providing the keyname at the end of the dictionary:

        Max_sal = Emp.objects.aggregate(Max('Sal'))['Sal__max']
#### annotate
* annotate method is used to create a temporary column for the execution purpose using aggregate functions
* Example is to display the rows that has 5 or more characters in EName 
* Syntax:

        LEO = Emp.objects.annotate(NL = Length('EName')).filter(NL__gte = 5).values_list('EName')

* values_list is used to get the ENanes directly from the queryset of Emp objects

#### Implementing 'Group by' using Annotate: To find Avg Sal in each Dept

    Emp.objects.values('Dept_No').annotate (Avg('Sal'))

#### Updating the records:
In 2 ways, we can update the records

    1. update_or_create method
    2. update method
#### Syntax for updating a particular row

    model_name.objects.filter(condition).update(column_name = 'value')
#### Syntax for updating entire table

    model_name.objects.all().update(column_name = value1,....)  
#### Note
1. Update command will update the rows which satifies the given condition
2. If no rows are satisying the condition, then it will not do any operation
3. Updates multiple rows as well
4. While dealing with foreign key column, we need to provide values which are present in the main/parent table

#### update_or_create
Syntax:

    model_name.objects.update_or_create(column_name = value,..., defaults = {'column_name': 'value'})

#### Note:
1. If a row satisfies the condition, then it gets updated
2. Incase, if such a row doesnot exist, then a new record is created
3. Mutliple records can not be created or updated
4. While dealing with foreign key column, we need to provide instance which is present in the main table

#### Deleting the records from the table
We use delete method inorder to delete data from the tables

Syntax for deleting a particular record/row from the table:

    model_name.objects.filter(condition).delete()

SQL syntax:   delete from table_name where condition

Syntax for deleting entire data from a model:

    model_name.objects.all().delete()   
                OR
    model_name.objects.delete()

SQL syntax:  truncate table table_name